**SOW-MKI49: Neural Information Processing Systems**  
*Weeks 2 and 3: Assignment (200 points + 20 bonus points + 1 bonus point for each bug you find and another bonus point if you debug it and before you ask, no, typos unfortunately are not considered bugs - first come, first served)*  
Author: Umut

In [ ]:
# Group number: 14
# Veerle Schepers s1023102
# Angeliki-Ilektra Karaiskou, s1029746
# Lei Xiaoxuan, s1025681

In [ ]:
batch_size = 4
data_directory = 'C:\\Users\\Αngeliki-Ilektra\\NeuralInformationProcessingSystems\\1stAs\\Data_directory'
                    # Make a directory to store the data and enter it here.
                    # We will be using a smaller dataset (LFW) than the one used in the paper (CelebA) for computational resource considerations.
                    # Download it from http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz.
device = -1
epochs = 4
lambda_ = {'feature': 1., 'pixel': 1., 'total_variation': 1e-5}
model_directory = 'C:\\Users\\Αngeliki-Ilektra\\NeuralInformationProcessingSystems\\1stAs\\modelDirectory'
# Make a directory to store the models and enter it here. Move Vgg4Layers.npz to the model directory.
outsize = (96, 96)

**Packages (10 points)**  
In this cell, you will import the required packages.  
*Tasks*   
- (1) It is always good practice to first think about the big picture and not rush into writing code before clearly knowing everything that you will have to do so as to avoid future complications. Therefore, your first task is to study the skeleton code and come up with a plan of how to proceed. (**0 points**)
- (2) However, I agree that doing so is arguably the most boring part of coding, and you rather skip it. To help you to resist the temptation of skipping going through the skeleton code, I have removed the import statements. Your second task is to Identify the required packages and import them. Note that if you are using Python 2.7, you should import print from the future. (**10 points**)

In [ ]:
# (2) start
import numpy as np
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

from PIL import Image, ImageFilter
from resizeimage import resizeimage

# (2) end

import os
import shutil
import cv2

from chainer.optimizers import Adam
from chainer.backends import cuda
import glob
from glob import glob
from chainer.serializers import load_npz, save_npz
from pprint import pprint
from chainer.dataset import concat_examples
from matplotlib import pyplot as plt
from chainer.dataset import DatasetMixin

#skiimage package
import matplotlib.pyplot as plt

from skimage import data, img_as_float

#psnr
from skimage.measure import compare_psnr as psnr
#ssim
from skimage.measure import compare_ssim as ssim

**Preprocessing functions (10 points + 5 bonus points)** (taken from https://github.com/mbeyeler/opencv-python-blueprints)  
In the following cell, you will implement some of the preprocessing functions. The rest of the preprocessing steps have already been applied to the data.  
*Tasks*
- (1) Implement the resizing operation. That is, you should extract the data, resize each portrait to 96 pixels x 96 pixels and save them to the data directory as JPG. (**10 points **)
- (2) The pencil sketch class implements the sketch effect in a simpler way than the one mentioned in the lecture. Explain how/why the used operations (blur and divide) convert portraits to sketches, and how it differs from that which was mentioned in the lecture? (**5 bonus points**)

In [ ]:
# (1) start resize image
# create a new folder to store the resized images
dirName = './Data_directory/Resize'


# If the folder already exist (from a previous running), delete the folder and it's content
if os.path.exists(dirName):
    shutil.rmtree(dirName)
else:    ## Show an error ##
    print('Error')
os.makedirs(dirName)   


# list all the names of the folder in lfw_deepfunneled
data_f=os.listdir(data_directory + '/lfw_deepfunneled')
#print data_f

# create path 
path = os.path.join(data_directory, 'lfw_deepfunneled')
#print path

# for every indivdual map
for img_f in data_f:
    # create path to, and a list of all the folders in lfw_deepfunneled
    path2 = os.path.join(path, img_f);
    data_f2 = os.listdir(path2)
    #print data_f2
    
    # for every individual image
    for img_f2 in data_f2:
        # create path to, and a list of all the images in the personal map
        path3 = os.path.join(path,img_f, img_f2)
        Parh3 = path3.replace('\\', '/')
        # print path3
        
        r_img = Image.open(path3)
        #r_img.show()
        r_img = r_img.resize((96, 96), Image.ANTIALIAS)
        r_img.save(data_directory+ '/resize_'+ img_f2);
# (1) end'''

class PencilSketch:
    """Pencil sketch effect
        A class that applies a pencil sketch effect to an image.
    """

    def __init__(self, dimension):
        """Initialize parameters
            :param (width, height): Image size.
        """
        self.width,self.height=dimension
        


    def render(self, img_rgb):
        """Applies pencil sketch effect to an RGB image
            :param img_rgb: RGB image to be processed
            :returns: Processed RGB image
        """
        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        img_blur = cv2.GaussianBlur(img_gray, (21, 21), 0, 0)
        img_blend = cv2.divide(img_gray, img_blur, scale=256)

        # return cv2.cvtColor(img_blend, cv2.COLOR_GRAY2RGB)
        return img_blend

def pencil_sketch(img_rgb):
    pencilSketch = PencilSketch((img_rgb.shape[1], img_rgb.shape[0]))

    return pencilSketch.render(img_rgb)

# (2) The operation first converst the colored image to a grayscale image to work on one layer
# In the next step, this gray image is blurred. This creates a smooth image with a low amount of noise
# Dividing the grayscale image by the blurred images rusults in an image in which only the parts
# where the original image differs much from the blurred images, which is a the edges, will
# be present. This gives a line sketch result.
# In the lecture a method in which 3 different skatches, namely a linesketch, grayscale sketch,
# and color sketch are created. 
# For the method mentioned in the paper is based on Gaussian kernal but for pencilSketch from cv2 
# is based on bilateral filtering

def preprocess(img):
    if img.mode == 'L':
        return np.rollaxis(np.asarray(img, 'float32')[..., None], 2)
    else:
        return np.rollaxis(np.asarray(img, 'float32')[..., ::-1], 2)

**Data class**  
The following cell defines the data class. It is used to manage the data (loading, etc.). *You do not have to make any changes to the code.*  
*Task*
- (1) Study the code and refer to the chainer docuimentation if anything is unclear. You will be expected to write similar code by yourself in the coming practicals. (**0 points**)

In [ ]:
class Dataset(DatasetMixin):
    def __init__(self, data_files):
        self.data_files = data_files

    def __len__(self):
            return len(self.data_files)

    def get_example(self, i):
        t = np.asarray(Image.open(self.data_files[i]).convert('RGB').resize((96, 96), Image.LANCZOS), 'f').transpose(2, 0, 1)

        x = pencil_sketch(np.asarray((Image.open(self.data_files[i]).convert('RGB').resize((96, 96), Image.LANCZOS)), 'f'))[None]

        return t, x

**Model classes (45 points)**  
In the following cellyou will implement the model classes.
*Tasks*   
- (1) Implement the layers of the model by filling in the missing code. (**20 points**)
- (2) Reimplement the model as a ChainList instead of a Chain. (**5 points**)
- (3) Implement the forward pass of the residual block by filling in the missing code. (**20 points**)

In [ ]:
class Model(Chain):
    def __init__(self, in_channels, outsize):
        super(Model, self).__init__()

        with self.init_scope():
            convolution2D_0=L.Convolution2D(1,32,9,pad = 4, nobias = True)
            batchNormalization_0 = L.BatchNormalization(32)
            convolution2D_1 = L.Convolution2D(32,64,3, stride=2, pad=1,nobias=True)
            batchNormalization_1 = L.BatchNormalization(64)
            convolution2D_2 = L.Convolution(64,128,3,stride=2,pad=1,nobias=True)
            batchNormalization_2 = L.BatchNormalization(128)
            residualBlock_3 = ResidualBlock(3,128,128,1)
            residualBlock_4 = ResidualBlock(3,128,128,1)
            residualBlock_5 = ResidualBlock(3,128,128,1)
            residualBlock_6 = ResidualBlock(3,128,128,1)
            residualBlock_7 = ResidualBlock(3,128,128,1)
            deconvolution2D_9 = L.Deconvolution2D(128, 64, 3, 2, 1, True, outsize)
            batchNormalization_9 = L.BatchNormalization(64)
            deconvolution2D_10 = L.Deconvolution2D(64, 32, 3, 2, 1, True, outsize)
            batchNormalization_10 = L.BatchNormalization(32)
            convolution2D_11 = L.Convolution2D(32, 3, 9, pad = 4, nobias = True)
            batchNormalization_11 = L.BatchNormalization(3)

        self.in_channels = in_channels
        self.outsize = outsize

    def __call__(self, x, finetune = False):
        h = self.convolution2D_0(x)
        h = self.batchNormalization_0(h, finetune)
        h = F.relu(h)
        h = self.convolution2D_1(h)
        h = self.batchNormalization_1(h, finetune)
        h = F.relu(h)
        h = self.convolution2D_2(h)
        h = self.batchNormalization_2(h, finetune)
        h = F.relu(h)
        h = self.residualBlock_3(h, finetune)
        h = self.residualBlock_4(h, finetune)
        h = self.residualBlock_5(h, finetune)
        h = self.residualBlock_6(h, finetune)
        h = self.residualBlock_7(h, finetune)
        h = self.deconvolution2D_8(h)
        h = self.batchNormalization_8(h, finetune)
        h = F.relu(h)
        h = self.deconvolution2D_9(h)
        h = self.batchNormalization_9(h, finetune)
        h = self.deconvolution2D_10(h)
        h = self.batchNormalization_10(h, finetune)
        h = F.relu(h)
        h = self.convolution2D_11(h)
        h = self.batchNormalization_11(h, finetune)
        y = 127.5 * F.tanh(h) + 127.5

        return y

class Model(ChainList):
    def __init__(self,in_channels,outsize):
    #def __init__(self):
        super(Model, self).__init__(
            L.Convolution2D(1,32,9,pad = 4, nobias = True), #0
            L.BatchNormalization(32), #1
            L.Convolution2D(32,64,3,stride = 2, pad=1,nobias=True), #2
            L.BatchNormalization(64), #3
            L.Convolution2D(64,128,3,stride = 2, pad=1,nobias=True), #4
            L.BatchNormalization(128), #5
            ResidualBlock(128,128), #6
            ResidualBlock(128,128), #7
            ResidualBlock(128,128), #8
            ResidualBlock(128,128), #9
            ResidualBlock(128,128), #10
            L.Deconvolution2D(128, 64, 3, 4, 1, True, outsize), #11
            L.BatchNormalization(64), #12
            L.Deconvolution2D(64, 32, 3,1, 1, True, outsize), #13
            L.BatchNormalization(32), #14
            L.Convolution2D(32, 3, 9, pad = 4, nobias = True), #15
            L.BatchNormalization(3), #16     
        )
        
    def __call__(self, x):
        h = self[0](x)
        h = self[1](h)
        h = F.relu(h)
        h = self[2](h)
        h = self[3](h)
        h = F.relu(h)
        h = self[4](h)
        h = self[5](h)
        h = F.relu(h)
        h = self[6](h)
        h = self[7](h)
        h = self[8](h)
        h = self[9](h)
        h = self[10](h)
        h = self[11](h)
        h = self[12](h)
        h = F.relu(h)
        h = self[13](h)
        h = self[14](h)
        h = F.relu(h)
        h = self[15](h)
        h = self[16](h)
        y = 127.5 * F.tanh(h) + 127.5

        return y

class ResidualBlock(Chain):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()

        with self.init_scope():
            self.batchNormalization_0 = L.BatchNormalization(out_channels)
            self.batchNormalization_1 = L.BatchNormalization(out_channels)
            self.convolution2D_0 = L.Convolution2D( in_channels, out_channels, 3, pad = 1, nobias = True)
            self.convolution2D_1 = L.Convolution2D(out_channels, out_channels, 3, pad = 1, nobias = True)

        self.in_channels = in_channels
        self.out_channels = out_channels

    def __call__(self, x, finetune = False):
    
        h = self.convolution2D_0(x)
        h = self.batchNormalization_0(h)
        h = F.relu(h)
        h = self.convolution2D_1(h);
        h = self.batchNormalization_1(h);
        h = F.relu(h)
        y = h + x

        return y

**Loss classes (45 points)**  
In the following cell, you will implement the loss classes.  
*Tasks*  
- (1) You are provided with a custom VGG-16 implementation. How does it differ than the original implementation? Why can we get away with using the simpler implementation? (**5 points**)
- (2) Implement the missing convolution layer of the total variation loss by filling in the missing code. (**10 points**)
- (3) Implement the forward pass of the total variation loss by filling in the missing code. (**10 points**)
- (4) Implement the feature loss component in the forward pass of the loss function by filling in the missing code. (**10 points**)
- (5) Explain why the loss components are scaled. (**5 points**)
- (6) Explain why the target features are extracted in test mode. (**5 points**)

In [ ]:
class Vgg4Layers(Chain):
    def __init__(self):
        super(Vgg4Layers, self).__init__()

        with self.init_scope():

            
            self.conv1_1 = L.Convolution2D( 3, 64, 3, pad = 1)
            self.conv1_2 = L.Convolution2D( 64, 64, 3, pad = 1)
            self.conv2_1 = L.Convolution2D( 64, 128, 3, pad = 1)
            self.conv2_2 = L.Convolution2D(128, 128, 3, pad = 1)
            

        #add_persistent('mean', np.array([[[[103.939]], [[116.779]], [[123.68]]]]), 'float32')
        self.mean = np.array([[[[103.939]],[[116.779]],[[ 123.68]]]], dtype='float32')
        #self.add_persistent('mean', np.array([[[[103.939]], [[116.779]], [[123.68]]]]), 'float32')

        
        
    def __call__(self, x):
        print(type(x))
        print("x.shape =")
        print(x.shape)
        h = x - F.broadcast_to(self.mean, x.shape) 
        
        h = self.conv1_1(h)
        h = F.relu(h)
        h = self.conv1_2(h)
        h = F.relu(h)
        h = F.max_pooling_2d(h, 2, 2)
        h = self.conv2_1(h)
        h = F.relu(h)
        h = self.conv2_2(h)
        y = F.relu(h)

        return y

    
    
class TotalVariationLoss(Chain):
    def __init__(self):
        super(TotalVariationLoss, self).__init__()

        with self.init_scope():
            self.convolution2D_0 = L.Convolution2D(3, 1, 2, nobias = True, initialW = np.array([3 * [[[-1], [1]]]], 'float32'))
            # (2) start
            self.convolution2D_1 = L.Convolution2D(3, 1, 2, nobias = True, initialW = np.array([3 * [[[1, -1]]]], 'float32'))
            # (2) end

    def __call__(self, x):
        
        # (3) start
        h0=np.square(self.convolution2D_0(x))
        h1=np.square(self.convolution2D_1(x))
        
        h=np.sum((h0,h1))
        y=h**0.5
        
        # y = ...
        # (3) end
        print (y)

        return y


    
class LossFunction(object):
    def __init__(self, lambda_):
        self.totalVariationLoss = TotalVariationLoss()
        self.vgg4Layers         = Vgg4Layers()

    def __call__(self, t, y):            
        with chainer.using_config('train', False):
            t_ = self.vgg4Layers(t)
        
        # (4) start
        
            y_ = self.vgg4Layers(y)
            feature_loss=lambda_['pixel']*F.mean_squared_error(t, y)
        # ....
        # (4) end
        pixel_loss = lambda_['pixel'] * F.mean_squared_error(t , y)
        total_variation_loss = lambda_['total_variation'] * self.totalVariationLoss(y)
        loss = feature_loss + pixel_loss + total_variation_loss
        
        print(loss)
        

        return loss


# (1) VGG4 is using only 4 layers in contrast with the VGG16 which uses 16 layers.
#     Because the CNN are extracting features hierarchically, the more layers are used the more
#     abstract "concepts" the features will represent. So by using more layers the pixel-loss is computing
#     and pixel-loss can not "capture perceptual differences between output and ground-truth images"(Johnson-Alahi-FeiFei)..
#     So by using the feature-loss functions, based on differences between high-level image feature 
#     representations extracted from pretrained convolutional neural networks, results to the production
#     of higher quality images (Johnson-Alahi-FeiFei)

# (5) Every loss component is representing a different measure, so in order to be added they have to represent 
#     the same quantity. That is being feasible with the scaling
# (6) We don't understand the question

**Initialization (10 points)**  
The following cell initializes the loss function, the loss history, the model, the optimizer, the datasets and the iterators. *You do not have to make any changes to the code.*  
*Tasks*
- (1) Study the code and refer to the chainer docuimentation if anything is unclear. You will be expected to write similar code by yourself in the coming practicals. (**0 points**)  
- (2) What are the boolean arguments that are passed to the SerialIterator class? (**5 points**)  
- (3) Why is it false for the training iterator but not for other iterators? In other words, what would happen if we were to set it to false for the training iterator and true for the other iterators? (**5 points**)

In [ ]:
lossFunction = LossFunction(lambda_)
serializers.load_npz('{:s}/Vgg4Layers.npz'.format(model_directory), lossFunction.vgg4Layers)
loss_history = {'training': [], 'validation': [],'test':[]}
model = Model(3, outsize) if device < 0 else Model(3, outsize).to_gpu(device)
optimizer = Adam()

optimizer.setup(model)




data_file = sorted(glob('{}/*.jpg'.format(data_directory)))

int1 = int(.64 * len(data_file))
int2 = int(.8 * len(data_file))
dat1 = data_file[:int1]
dat2 = data_file[int1:int2]
dat3 = data_file[int2:]


#training_set = Dataset(data_file[: int( .64 * len(data_file))])
#validation_set = Dataset(data_file[int(.64 * len(data_file) : .8 * len(data_file))])
#test_set = Dataset(data_file[int(.8 * len(data_file) :)])
training_set = Dataset(dat1)
validation_set = Dataset(dat2)
test_set = Dataset(dat3)

training_iterator = iterators.SerialIterator(training_set, batch_size, False, True)
validation_iterator = iterators.SerialIterator(validation_set, batch_size, False, False)
test_iterator = iterators.SerialIterator(test_set , batch_size, False, False)

# (2) Booleans are always either true or false. 
# The first boolean is called repeat. 
# If it is set to true, it infinetely loops over the dataset (training_set, validation_set, or test_set)
# If it is set to false, it stops iteration at the end of the first epoch.  

# The second boolean is called shuffle. 
# If it is set to true, at the beginning of each epoch the examples are shuffled.
# If it is set to false, the examples are extracted in the order of indexes. 

# (3) The second boolean is True for the training iteratior and false for the other iterators. 
# Shuffle is True during the training process to make sure that the model does not overfit to the examples.
# Shuffle is False during the other processes to be able to reproduce results.
# If we would change this, unintentional performance degradtion will occure.



**Training and validation (20 points)**  
In the following cell, you will train and validate your model.
*Tasks*   
- (1) Implement training loss estimation, backprop and parameter update. (**10 points**)
- (2) Implement validation loss history (**5 points**)
- (3) Implement model serialization  (**5 points**)

In [ ]:
for i in range(epochs):
    loss_history['training'].append(0)

    for j, batch in enumerate(training_iterator):
        with chainer.using_config('train', True): #Chainer runs in trainig mode
            t, x = concat_examples(batch, device)
            y = model(x)
            # (1) start  
            
            y.data
            
            
            # training loss estimation
            print("Values before")
            loss = lossFunction(t, y)
            print("Values afterwards")
            
            
            # backprop
            model.cleargrads()
            loss.backward()
            
            
            #derivative = y * (1.0 - y)
            #def transfer_derivative(y):
            #    return output * (1.0 - y)
            
            # parameter update
            optimizer.update() 
            
            
            # (1) end

        loss_history['training'][-1] += float(loss.data)

    loss_history['training'][-1] /= j + 1
    # (2) start
    # ...
    loss_history['validation'].append(0)
    

   for j, batch in enumerate(validation_iterator):
        with chainer.using_config('train', False):
            t, x = concat_examples(batch, device)
            y = model(x)
            loss = lossFunction(t, y)
            
        loss_history['validation'][-1] += float(loss.data)
    loss_history['validation'][-1] /= j + 1

    # ...
    # (2) end
    print('epoch: {:3d} / {:03d}, training loss: {} , validation loss: {}.'.format(i + 1, epochs, loss_history ['training'], loss_history['validation']))
    np.savez('{:s}/loss_history_{:03d}.npz'.format(model_directory, epoch), loss_history)
    # (3) start
    serializers.save_npz('our.model', model)
    
    # (3) end
    save_npz('{:s}/optimizer_{:03d}.npz'.format(model_directory, epoch), optimizer)

**Test (45 points + 15 bonus points)**  
In the following cell, you will test your model.  
*Tasks*
- (1) Estimate the test loss, print it and save it. (**15 points**)
- (2) Estimate the validation metrics, print them and save them (tip: scikit-image) (**15 bonus points**)
- (3) Plot example results (i.e., plot a few t, x and y) (**10 points**)
- (4) Dicuss your implementation in 300 - 350 words (e.g., how good your results are, how you can improve your model, etc.) (**20 points**)

In [ ]:
# (1), (2) and (3) start
# (1) start
lossFunction=LossFunction(lambda_)
serializers.load_npz('{:s}/Vgg4Layers.npz'.format(model_directory), lossFunction.vgg4Layers)
loss_history['test'].append(0)
#ourModel=Model(3, outsize)
ourModel = Model(3, outsize) if device < 0 else Model(3, outsize).to_gpu(device)

serializers.load_npz('{:s}/our.model'.format(model_directory),ourModel)
print(ourModel)
type(ourModel)

for k, batch in enumerate(test_iterator):
    with chainer.using_config('test', False):
        print("-----------------------------------------")
        print(" k = ")
        print(k)
        t, x = concat_examples(batch, device)
        t = np.array(t)
        y = ourModel(x)
        loss = lossFunction(t, y)
    loss_history['test'][-1]+=float(loss.data)
    # PSNR Metric
    PSNR=psnr(t,y.data,data_range=255)
    print('PSNR=')
    print(PSNR)
    #imshow(PSNR)
    # SSIM Metric
    #SSIM=ssim(t, y.data,data_range=None,multichannel=True)
    #print('SSIM=')
    #print(SSIM)
    #imshow(PSNR)
loss_history['test'][-1]/=k+1
print(loss_history['test'][-1])
np.savez('{:s}/test_loss_history.npz'.format(model_directory), loss_history['test'])
# (1) end 
# (1), (2) and (3) end

# (1), (2) and (3) start
# .
# .
# .
# (1), (2) and (3) end

# (4) 
#   Our model is not performing that well and therefore it could be improved very much. 
#    Our PSNR results differ between 12 and 17, while the PSNR of the convolution sketch 
#    inversion paper goes up to over 20. Another thing that has to be looked at is that 
#    the difference in our PSNR results is much larger than 0.0231 bootstrap estimate of 
#    the standard error of the mean in the paper. We are not surprised of our results and 
#    have some ideas of how we could improve the model. To begin with, we only used a very 
#    small subset of the LFW dataset which is already a subset of the CelebA dataset that 
#    is used for the model discussed during the lectures and in the paper. The LFW dataset 
#    consist of over 13.000 images, of which we only used 200. We had no other choice than 
#    taking this small subset since, as far as we know, there are not online GPU’s that can 
#    work with chainer. Therefore, we had to train the model on one of our laptops and this 
#    takes, as would be no surprise, very long. After the deadline we will try to train our 
#    model with the entire dataset, to see how much it will improve.
#    Another way in which our model could be improved is by using more sketches types. For 
#    the model discussed in the lecture three sketch types are used, namely a line sketch, 
#    a gray scale sketch, and a color sketch. In our model we only used the line sketch, 
#    which was the way to go for this assignment. The amount of improvement caused by using 
#    multiple types of sketches is off course very small compared to the improvement that would 
#    be gained by using 65 times the amount of images we used. Furthermore, after we run the model 
#    with the complete LFW dataset, it would be more interesting to look at the results of the model 
#    and from these results we might could see more ways in which our model could be improved.

In [ ]:
plt.imshow(y, cmap=plt.cm.Greys)